In [1]:
import pandas as pd
import numpy as np
import os
%pprint off

Pretty printing has been turned OFF


In [2]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [3]:
fpath = '/Users/songkim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/results_clean_0628.csv'

In [76]:
res = pd.read_csv(fpath, converters={'response': eval, 'RT': eval, 'buttonpress': eval, 'catch_response': eval})
res.sort_values(by=['stim_id', 'id'], inplace=True)
print('before catch-based rejection:', res.shape)

######################### check number of responses per lemma
# for i in set(res['stim_id']): 
#     df = res.loc[res['stim_id']==i]
#     print (i, df.shape[0])

########################## convert N/A to 0 
res['catch_response_upd'] = res['catch_response'].apply(lambda x: [0 if i=='7' else float(i) for i in x])
res['response_upd'] = res['response'].apply(lambda x: [0 if i=='7' else float(i) for i in x])

res = rearrange_col(res, 'catch_response_upd', 'catch_response')
res = rearrange_col(res, 'response_upd', 'response')

res.drop(columns=['catch_response', 'response'], inplace=True)
res.rename(columns={'response_upd': 'response', 'catch_response_upd':'catch_response'}, inplace=True)

# ######################### drop bad subjects
res = res.loc[res['badsubject']=='N', ]
print ('after catch-based rejection:', res.shape)
res.head()

before catch-based rejection: (2816, 20)
after catch-based rejection: (2458, 20)


,id,stim_id,stim,subject_id,response,start,end,RT,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response,correctN,badsubject
213,214,1,live,206,"[6.0, 4.0, 1.0, 3.0, 2.0, 6.0, 2.0, 6.0, 6.0, ...",2021-05-26 11:38:39,2021-05-26 11:46:03,"[2253, 6104, 4009, 3299, 5748, 9628, 5439, 284...","[1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4940126,A1ZDR2UN7M92LI,m,37,16.0,Foodprep,214,206,"[0.0, 6.0, 0.0, 6.0, 6.0]",4,N
220,221,1,live,213,"[3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 4.0, ...",2021-05-26 11:35:37,2021-05-26 11:49:09,"[8232, 1864, 2304, 1760, 3624, 3744, 2015, 284...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...",4019101,ALOYY3E72AQ1K,f,56,16.0,digitalproductmanagerworkingononlinecontentfor...,221,213,"[0.0, 6.0, 0.0, 6.0, 6.0]",4,N
231,232,1,live,223,"[2.0, 2.0, 0.0, 2.0, 1.0, 0.0, 0.0, 5.0, 5.0, ...",2021-05-26 10:54:50,2021-05-26 11:53:10,"[19852, 7922, 172288, 4293, 73124, 2751, 11801...","[1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",9110817,A171S8E9IFSHH2,m,33,14.0,healthcaresupport,232,223,"[0.0, 6.0, 0.0, 6.0, 6.0]",4,N
840,841,1,live,286,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 5.0, ...",2021-05-30 07:16:46,2021-05-30 07:26:46,"[14411, 3015, 1406, 5198, 1980, 2520, 4139, 28...","[1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, ...",4202983,A1EJ4HA4K85PBJ,f,53,18.0,communicationsdirectorforagovernmentagency,287,286,"[0.0, 6.0, 0.0, 6.0, 6.0]",4,N
1095,1096,1,live,266,"[2.0, 1.0, 0.0, 0.0, 0, 0.0, 0.0, 4.0, 6.0, 2....",2021-06-03 17:58:11,2021-06-03 18:06:46,"[12905, 5097, 2779, 3240, 5073, 3374, 2363, 44...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",8140924,A1AZLJ3R5FQ9B7,f,67,16.0,Selfemployedwriter,542,266,"[0.0, 6.0, 0.0, 6.0, 6.0]",4,N


In [83]:
#################### create an average dict ###################### 
stim_id_n = len(set(res['stim_id']))  ###i.e., 320

response_n = len(res.iloc[0]["response"])
column = []
for i in range(1, response_n+1):
    column.append('q'+str(i))  ##### column = ['q1', 'q2', ... 'q72']

avg_dict = {}
for i in range(1, stim_id_n+1):
    df = res.loc[res['stim_id']==i, ]
    df_responses = pd.DataFrame(df.response.tolist(), columns=column).astype(int)
    avg = list(df_responses.mean())
    avg = [round(i,3) for i in avg]
#     print (df.shape)
#     print (df_responses.shape)

    avg_dict[i] = {'stim': df.iloc[0]['stim'], 'subject_ids': df['subject_id'].values.tolist(), 
                  'response_N': df.shape[0], 'response_avg': avg}
#avg_df = pd.DataFrame.from_dict(avg_dict, orient='index')

In [84]:
avg_dict[1]

{'stim': 'live', 'subject_ids': [206, 213, 223, 286, 266, 179, 5, 97, 123], 'response_N': 9, 'response_avg': [2.333, 1.111, 0.222, 0.778, 0.444, 0.778, 0.778, 3.111, 4.667, 1.667, 1.889, 2.111, 2.667, 4.0, 1.667, 0.556, 0.333, 0.444, 0.778, 0.222, 0.111, 1.556, 2.0, 0.556, 0.222, 0.667, 0.889, 0.778, 3.111, 1.333, 1.0, 2.667, 2.889, 2.444, 5.667, 2.667, 1.778, 2.111, 0.667, 0.667, 0.667, 0.556, 2.0, 3.222, 4.333, 2.111, 3.222, 4.111, 4.0, 5.667, 2.778, 5.444, 4.333, 4.0, 1.0, 4.889, 0.444, 3.667, 1.0, 0.889, 0.333, 1.0, 0.889, 4.0, 5.333, 2.111, 3.667, 2.778, 3.444, 0.889, 4.111, 3.778]}

In [89]:
########calculate intersubject correlation and update res
from scipy.stats.stats import pearsonr

res_dict = res.set_index('id').to_dict(orient='index')
for k, v in res_dict.items():
    response = v['response']
    group_response = avg_dict[v['stim_id']]['response_avg']
    v['corr'] = round(pearsonr(response, group_response)[0],3) ###correlation b/w this vector and group average vector for word X.

#pearsonr(response, group_response)[0]

### convert to a df
res_upd = pd.DataFrame.from_dict(res_dict, orient='index')

In [ ]:
########### TODO 
### get overall descriptive statistics: mean intra-word individual-to-group correlation: 
## pick out worst words: lower inter-subject correlation 

In [ ]:
# list_1 = [1, 1, 1, 1]
# list_2 = [-1, -1, -1, -1]
# list_3 = [1, 3, 5, 7]
# list_4 = [0,0,0,100]
# dfx=pd.DataFrame({"list_1":list_1,"list_2":list_2,"list_3":list_3,"list_4":list_4})
# dfx

In [ ]:
# t1= '0_6_0_6_6'
# t1.split('_')

# testlist = ['5', '7', '3', '0', '6', '0', '3', '1', '0']
# ['0' if t=='7' else t for t in testlist]
#new_prices = [round(price - (price * 10 / 100), 2) if price > 50 else price for price in prices]

In [ ]:
####### spread responses into separate columns
# df_test = res[["id", "stim_id", "response"]]

#print ('before spreading:', res.shape)

# response_n = len(res.iloc[0]["response"])
# column = []
# for i in range(1, response_n+1):
#     column.append('q'+str(i))  ##### column = ['q1', 'q2', ... 'q72']

# res[column] = pd.DataFrame(res.response.tolist(), index=res.index)

# print ('after spreading:', res.shape)

#res.tail()

#df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist(), index= df2.index)